# Bedrijven Data Analyse

## Overzicht
Deze notebook combineert en analyseert alle Tilburg bedrijfsdata uit verschillende scraping runs.

### Data Pipeline Overzicht:
1. **CSV Bestanden Laden**: Combineer alle beschikbare CSV bestanden (exclusief cleaned_data folder)
2. **Reeds Geëxporteerde Adressen Filteren**: Filter alle adressen uit cleaned_data folder uit om duplicaten te voorkomen
3. **Grote Ketens Filtering**: Filter grote ketens uit
4. **Duplicaten Verwijderen**: Verwijder duplicaten op basis van bedrijfsnaam
5. **Adres Filtering**: Filter op bedrijven met compleet adres (straat en Nederlandse postcode)
6. **Categorieën Analyse**: Analyseer de verdeling van bedrijfscategorieën en amenities
7. **Export**: Sla gefilterde data op voor gebruik in scraper

### Veiligheid
**BELANGRIJK**: Alle originele CSV bestanden blijven volledig onbeschadigd! De notebook leest alleen de bestanden en schrijft nooit terug naar de originele bestanden. Alle output wordt opgeslagen in de `cleaned_data/` folder.




In [1]:
# Import benodigde libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Stel matplotlib in voor betere visualisaties
plt.style.use('default')
sns.set_palette("husl")

print("Libraries geïmporteerd!")


Libraries geïmporteerd!


## 1. CSV Bestanden Laden en Combineren

### Wat gebeurt er hier?
We laden automatisch alle CSV bestanden uit de huidige directory (exclusief cleaned_data folder) en combineren ze tot één grote dataset. Elk bestand krijgt een `source_file` kolom zodat we kunnen zien waar elke rij vandaan komt.

**Veiligheid**: Originele bestanden worden alleen gelezen, nooit gewijzigd!


In [2]:
# Automatisch alle CSV bestanden vinden (exclusief cleaned_data folder)
csv_files = []
for file in glob.glob("*.csv"):
    if not file.startswith("cleaned_"):  # Exclude cleaned files
        csv_files.append(file)

# Sorteer bestanden op naam voor consistente volgorde
csv_files.sort()

print(f"Gevonden {len(csv_files)} CSV bestanden om te laden:")
for file in csv_files:
    print(f"  - {file}")

print(f"\nLaden van {len(csv_files)} CSV bestanden...")
print("BELANGRIJK: Originele CSV bestanden worden alleen gelezen, niet gewijzigd!")

# Lijst om alle dataframes op te slaan
all_dataframes = []

# Loop door alle CSV bestanden
for file in csv_files:
    try:
        print(f"Laden: {file}")
        df = pd.read_csv(file)
        df['source_file'] = file  # Voeg source file kolom toe
        all_dataframes.append(df)
        print(f"  - {len(df)} rijen geladen")
    except Exception as e:
        print(f"Fout bij laden van {file}: {e}")

print(f"\nTotaal {len(all_dataframes)} bestanden succesvol geladen")
print("Originele CSV bestanden blijven volledig onbeschadigd!")


Gevonden 0 CSV bestanden om te laden:

Laden van 0 CSV bestanden...
BELANGRIJK: Originele CSV bestanden worden alleen gelezen, niet gewijzigd!

Totaal 0 bestanden succesvol geladen
Originele CSV bestanden blijven volledig onbeschadigd!


In [3]:
# Combineer alle dataframes
if all_dataframes:
    print("=== COMBINEREN VAN ALLE DATAFRAMES ===")
    
    # Combineer alle dataframes
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    
    print(f"Gecombineerde dataset: {len(combined_df)} rijen")
    print(f"Aantal kolommen: {len(combined_df.columns)}")
    print(f"Kolommen: {list(combined_df.columns)}")
    
    # Toon verdeling per source file
    print("\n=== VERDELING PER BRONBESTAND ===")
    source_counts = combined_df['source_file'].value_counts()
    for source, count in source_counts.items():
        print(f"{source}: {count} bedrijven")
    
    print(f"\nTotaal unieke bedrijven: {combined_df['name'].nunique()}")
    
else:
    print("Geen CSV bestanden gevonden om te combineren!")


Geen CSV bestanden gevonden om te combineren!


## 1.5. Filteren van Reeds Geëxporteerde Adressen

### Wat gebeurt er hier?
We laden alle adressen uit de `cleaned_data/` folder en filteren deze uit de nieuwe analyse om duplicaten te voorkomen.

**Filter criteria:**
- ✅ Alle adressen uit cleaned_data worden geladen
- ✅ Deze adressen worden uitgefilterd uit de nieuwe analyse
- ✅ Alleen nieuwe bedrijven worden behouden


In [4]:
# Laad alle adressen uit cleaned_data folder en filter ze uit de nieuwe analyse
if 'combined_df' in locals():
    print("=== FILTEREN VAN REEDS GEËXPORTEERDE ADRESSEN ===")
    
    # Lijst om alle adressen uit cleaned_data op te slaan
    cleaned_data_addresses = set()
    
    # Zoek alle CSV bestanden in cleaned_data folder
    cleaned_data_path = Path('cleaned_data')
    
    if cleaned_data_path.exists():
        cleaned_csv_files = list(cleaned_data_path.glob('*.csv'))
        
        print(f"Gevonden {len(cleaned_csv_files)} CSV bestanden in cleaned_data folder")
        
        # Loop door alle cleaned_data bestanden
        for file in cleaned_csv_files:
            try:
                df = pd.read_csv(file)
                
                # Check of 'address' kolom bestaat
                if 'address' in df.columns:
                    # Voeg alle adressen toe aan de set (lowercase, zonder whitespace voor betrouwbare matching)
                    addresses = df['address'].dropna().astype(str).str.lower().str.strip()
                    cleaned_data_addresses.update(addresses)
                    print(f"  - {file.name}: {len(addresses)} adressen toegevoegd")
                else:
                    print(f"  - {file.name}: Geen 'address' kolom gevonden")
                    
            except Exception as e:
                print(f"Fout bij laden van {file.name}: {e}")
        
        print(f"\nTotaal {len(cleaned_data_addresses)} unieke adressen uit cleaned_data geladen")
    else:
        print("cleaned_data folder niet gevonden - geen filtering nodig")
    
    # Filter de nieuwe data om adressen uit te sluiten die al in cleaned_data zitten
    if cleaned_data_addresses:
        original_count = len(combined_df)
        
        # Maak een copy van combined_df om warning te voorkomen
        combined_df = combined_df.copy()
        
        # Normaliseer adressen voor matching (handle NaN values)
        combined_df['address_normalized'] = combined_df['address'].fillna('').astype(str).str.lower().str.strip()
        
        # Filter: behoud alleen bedrijven die NIET in cleaned_data_addresses zitten
        is_already_exported = combined_df['address_normalized'].isin(cleaned_data_addresses)
        
        print(f"\nOriginele dataset: {original_count} bedrijven")
        print(f"Adressen reeds geëxporteerd (worden uitgefilterd): {is_already_exported.sum()}")
        
        if is_already_exported.sum() > 0:
            print(f"\nVoorbeelden van uitgefilterde adressen:")
            excluded_samples = combined_df[is_already_exported]['address'].head(10)
            for i, address in enumerate(excluded_samples, 1):
                print(f"  {i:2d}. {address}")
        
        # Update combined_df: verwijder bedrijven met adressen die al geëxporteerd zijn
        combined_df = combined_df[~is_already_exported].copy()
        
        # Verwijder de normalized kolom
        combined_df = combined_df.drop('address_normalized', axis=1)
        
        print(f"\nNa filtering reeds geëxporteerde adressen: {len(combined_df)} bedrijven")
        print(f"Verwijderd: {original_count - len(combined_df)} duplicaten")
    else:
        print("\nGeen cleaned_data adressen gevonden - alle bedrijven worden behouden")
        
else:
    print("Geen gecombineerde data beschikbaar voor filtering!")


Geen gecombineerde data beschikbaar voor filtering!


## 2. Geografische Filtering - OPTIONEEL

### Wat gebeurt er hier?
We filteren de gecombineerde dataset op basis van Nederlandse postcode (alle locaties binnen scraping radius).

**Filter criteria:**
- ✅ Adres bevat Nederlandse postcode (4 cijfers + 2 letters)
- ✅ Alle steden/locaties binnen scraping radius worden behouden
- ❌ GEEN plaats-specifieke filter (werkt met alle steden)


In [5]:
# Run cell 6 manually! It filters out big chains before location filtering
if 'combined_df' in locals():
    print("=== FILTEREN GROTE KETENS ===")
    
    # Lijst van grote ketens om uit te sluiten
    grote_ketens = [
        'Albert Heijn', 'AH to go', 'Jumbo', 'Lidl', 'Aldi', 'Plus', 'Dirk', 'Hoogvliet', 'Spar',
        'Nettorama', 'DekaMarkt', 'Boni', 'Vomar', 'Jan Linders', 'Poiesz', 'EkoPlaza', 'Picnic',
        'Makro', 'Sligro', 'Hanos', 'ACTION', 'Action', 'HEMA', 'Blokker', 'Xenos', 'Big Bazar',
        'SoLow', 'Normal', 'Flying Tiger Copenhagen', 'Søstrene Grene', 'Dille & Kamille',
        'Gall & Gall', 'Mitra', 'GrapeDistrict', 'Henri Bloem', 'Kruidvat', 'Etos', 'Trekpleister',
        'DA', 'Holland & Barrett', 'ICI Paris XL', 'Rituals', 'Douglas', 'The Body Shop',
        'Yves Rocher', 'MediaMarkt', 'Coolblue', 'BCC', 'Expert', 'EP', 'Amac', 'Apple Store',
        'Alternate', 'PhoneHouse', 'GSMweb', 'Vodafone', 'KPN', 'Odido', 'Tele2', 'Youfone',
        'Lebara', 'H&M', 'H&M Home', 'C&A', 'Zara', 'Bershka', 'Pull&Bear', 'Massimo Dutti',
        'Stradivarius', 'Mango', 'Primark', 'WE Fashion', 'The Sting', 'Costes', 'America Today',
        'Open32', 'Sissy-Boy', 'G-Star RAW', 'Levi\'s Store', 'Tommy Hilfiger', 'Calvin Klein Underwear',
        'Scotch & Soda', 'Superdry', 'Esprit', 'Jack & Jones', 'Only', 'Vero Moda', 'Vila',
        'Name It', 'Selected', 'KIABI', 'Zeeman', 'Wibra', 'Shoeby', 'Bristol', 'Scapino',
        'Omoda', 'Nelson', 'Manfield', 'Sacha', 'Ziengs', 'Schuurman Schoenen', 'VanHaren',
        'Foot Locker', 'JD Sports', 'Snipes', 'Size?', 'Intersport', 'Daka Sport', 'Perry Sport',
        'Bever', 'Decathlon', 'ANWB', 'Runnersworld', 'IKEA', 'Leen Bakker', 'Kwantum', 'JYSK',
        'Beter Bed', 'Swiss Sense', 'Beddenreus', 'Goossens', 'Profijt Meubel', 'Henders & Hazel',
        'XOOON', 'Montèl', 'Pronto Wonen', 'TotaalBED', 'Auping Store', 'Tempur Store',
        'Hästens', 'Rivièra Maison', 'Casa', 'Trendhopper', 'Karwei', 'Gamma', 'Praxis',
        'Hornbach', 'Bauhaus', 'Hubo', 'Bouwmaat', 'PontMeyer', 'Toolstation', 'Praxis Tuincentrum',
        'Intratuin', 'GroenRijk', 'Ranzijn Tuin & Dier', 'Welkoop', 'Pets Place', 'Jumper',
        'Discus', 'Dobey', 'Avonturia', 'Intertoys', 'ToyChamp', 'LEGO Store', 'Bart Smit',
        'Primera', 'Bruna', 'AKO', 'The Read Shop', 'BoekenVoordeel', 'Paagman', 'Donner',
        'Jamin', 'Leonidas', 'Australian Homemade', 'Chocolate Company', 'Simon Lévelt',
        'Kaldi Koffie & Thee', 'Coffeecompany', 'Starbucks', 'Bagels & Beans', 'Doppio Espresso',
        'Anne&Max', 'Barista Cafe', 'Coffeelovers', 'BackWERK', 'De Beren', 'Loetje',
        'Happy Italy', 'Vapiano', 'Sumo', 'Shabu Shabu', 'SushiPoint', 'Sushi Time',
        'Sushito', 'Poké Perfect', 'The Poké Bar', 'Poke House', 'Wok To Go', 'Eazie',
        'Johnny\'s Burger', 'Five Guys', 'Smullers', 'Febo', 'Kwalitaria', 'McDonald\'s',
        'Burger King', 'KFC', 'Subway', 'Domino\'s', 'New York Pizza', 'Pizza Hut',
        'Papa John\'s', 'La Place', 'Bakker Bart', 'Délifrance', 'Dunkin\'', 'TGI Friday\'s',
        'Coffee Fellows', 'Starbucks Reserve', 'Brownies&downieS', 'Exki', 'Kiosk',
        'AH to go Stations', 'Julia\'s', 'HEMA Deli', 'Van der Valk', 'Fletcher Hotels',
        'Bastion Hotels', 'NH Hotels', 'Leonardo Hotels', 'Eden Hotels', 'Postillion Hotels',
        'Campanile', 'B&B Hotels', 'Ibis', 'Ibis Budget', 'Ibis Styles', 'Novotel', 'Mercure',
        'Hilton', 'DoubleTree by Hilton', 'Marriott', 'Moxy', 'Sheraton', 'Holiday Inn',
        'Holiday Inn Express', 'Crowne Plaza', 'Radisson Blu', 'Park Plaza', 'Motel One',
        'citizenM', 'easyHotel', 'Stayokay', 'a&o Hostels', 'Room Mate', 'Aparthotel Adagio',
        'Pathé', 'Vue', 'Kinepolis', 'JT Bioscopen', 'Holland Casino', 'Jack\'s Casino',
        'Fair Play Casino', 'Rabobank', 'ING', 'ABN AMRO', 'SNS', 'ASN Bank', 'RegioBank',
        'Triodos Bank', 'De Hypotheker', 'De Hypotheekshop', 'Van Bruggen Adviesgroep',
        'Huis & Hypotheek', 'Univé', 'Aegon Shop', 'ASR Servicepunt', 'Allsecur Servicepunt',
        'Ohra Servicepunt', 'Profile', 'KwikFit', 'Euromaster', 'Vakgarage', 'CarProf',
        'Autoservice Totaal', 'James Autoservice', 'Bosch Car Service', 'Autotaalglas',
        'Carglass', 'Halfords', 'BOVAG Pechhulp Servicepunt', 'ANWB Winkel', 'Stella Fietsen',
        'Amslod', 'Fietsvoordeelshop.nl', 'Profile Fietsspecialist', 'Bike Totaal', 'Mantel',
        'Basic-Fit', 'Fit For Free', 'SportCity', 'TrainMore', 'Anytime Fitness', 'Snap Fitness',
        'David Lloyd', 'HealthCity', 'Fit20', 'Curves', 'Rocycle', 'Saints & Stars',
        'Club Pellikaan', 'Shape All In', 'BBB health boutique', 'Shell', 'BP', 'Esso',
        'TotalEnergies', 'Q8', 'Texaco', 'Tango', 'TinQ', 'Tamoil', 'OK Olie', 'AVIA',
        'Argos', 'Firezone', 'PostNL Pakketpunt', 'DHL ServicePoint', 'UPS Access Point',
        'DPD Pickup Parcelshop', 'GLS ParcelShop', 'PostMasters', 'PakjeGemak', 'ING Servicepunt',
        'Kiala Punt', 'Mister Minit', 'Shoelia', 'Key Service', 'Schoenmakerij Hakky',
        'brainWash Kappers', 'Team Kappers', 'Cosmo Hairstyling', 'AMI Kappers', 'Kinki Kappers',
        'Salon B', 'Rob Peetoom', 'The Barberstation', 'Cut & Go', 'VIProxx Barbers',
        'Pearle Opticiens', 'Hans Anders', 'Specsavers', 'Eye Wish', 'eyes + more', 'Optiek XL',
        'BENU Apotheek', 'Mediq Apotheek', 'Service Apotheek', 'Boots Apotheek',
        'Alphega Apotheek', 'Etos Apotheek', 'BENU Servicepunt', 'DA Apotheek', 'Douglas Hair',
        'Prénatal', 'Baby-Dump', 'Babypark', 'Baby Tiener', 'Noppies', 'Tumble \'N Dry Store',
        'Hunkemöller', 'Livera', 'Intimissimi', 'Tezenis', 'Calzedonia', 'Victoria\'s Secret',
        'Havaianas Store', 'Swarovski', 'Pandora', 'Lucardi Juwelier', 'Siebel Juweliers',
        'Gassan Boutique', 'Bijou Brigitte', 'Claire\'s', 'Six', 'HEMA Juwelier',
        'Flying Tiger Cadeau', 'Søstrene Grene Home', 'Boekenvoordeel', 'Bruna Boeken',
        'AKO Krantenshop', 'Primera Tabak & Lotto', 'CIGO', 'Tabaronde', 'Gall & Gall XL',
        'AH Gall Shop-in-shop', 'SPAR City', 'SPAR Express', 'COOP Vandaag', 'PLUS Vandaag',
        'AH XL', 'Jumbo City', 'Dirk XL', 'Hoogvliet Versmarkt', 'DekaMarkt World of Food',
        'Ekoplaza Foodmarqt', 'Gelderlandplein Foodmarket', 'Foodmaker', 'Marqt (AH)',
        'Vishandel Koning (keten)', 'Slagerij Keurslager', 'Bakkerij \'t Stoepje',
        'De Echte Bakker', 'Australian Ice Cream', 'IJscuypje', 'Pinkberry', 'Yoghurt Barn',
        'Van Uffelen', 'Peek & Cloppenburg', 'De Bijenkorf', 'TK Maxx', 'Sissy-Boy Homeland',
        'H&M Home Store', 'Zara Home Store', 'Normal Store', 'SoLow Party', 'Party City NL',
        'Partyland', 'Rituals Cosmetics', 'KIKO Milano', 'NYX Professional Makeup', 'Flormar',
        'MAC Cosmetics', 'Sephora', 'Douglas Pro', 'HEMA Beauty', 'Etos Beauty',
        'Bol.com Afhaalpunt', 'Beter Bed Compact', 'Swiss Sense Sleep Store', 'Beter Horen',
        'Van Boxtel Hoorwinkels', 'Schoonenberg HoorSupport', 'Castelijn Hoorstores',
        'Specsavers Hoorzorg', 'Pearle Hoorzorg', 'TUI', 'Sunweb Servicepunt', 'D-reizen',
        'VakantieXperts', 'ANWB Reizen', 'Corendon Servicepunt', 'Kras Reizen Servicepunt',
        'Prijsvrij Reizen Shop', 'NRV Servicepunt', 'Shoeby Kids', 'TerStal', 'Norah',
        'VanHaren Kids', 'Sacha Premium', 'Manfield Premium', 'Gabor Store', 'Ecco Store',
        'Birkenstock Store', 'Timberland Store', 'Dr. Martens Store', 'Levi\'s Tailor Shop',
        'Nike Store', 'Adidas Originals Store', 'PUMA Store', 'New Balance Store',
        'Under Armour Factory House', 'JD Flagship', 'Foot Locker House of Hoops',
        'Snipes Premium', 'The Athlete\'s Foot', 'CyclingDeal Store', 'Intersport Twinsport',
        'Jumbo Golf & Hockey', 'Hockey Republic', 'Runnersworld XL', 'Bever Zwerfkei',
        'Campz Store', 'Duifhuizen', 'Travelbags Store', 'Koffer Store', 'Suitsupply',
        'Suitable', 'Michael Kors', 'Coach Store', 'Longchamp', 'Furla', 'Hugo Boss',
        'BOSS Outlet', 'Tommy Hilfiger Tailored', 'Ralph Lauren Store', 'Massimo Dutti Men',
        'Massimo Dutti Women', 'COS', '& Other Stories', 'Arket', 'Weekday', 'Monki',
        'Uniqlo', 'Desigual', 'Superdry Store', 'G-Star Outlet', 'Scotch & Soda Outlet',
        'Outlet Roermond Designer', 'Batavia Stad Brandstores', 'VanHaren Outlet',
        'Nelson Outlet', 'Goossens Outlet', 'Leen Bakker Outlet', 'Swiss Sense Outlet',
        'Praxis Megastore', 'Gamma XL', 'Karwei Design', 'Hornbach Bouwdomein',
        'Bauhaus Bouwcentrum', 'Toolstation Express', 'Hubo Compact', 'Bouwmaat City',
        'PontMeyer Drive-in', 'Formido', 'Bo-Rent', 'Avis', 'Budget', 'Hertz', 'Sixt',
        'Enterprise', 'Europcar', 'Green Motion', 'Autohopper', 'Sunny Cars Servicepunt',
        'Peter Langhout Servicepunt', 'Basic Fit Ladies', 'Anytime Fitness 24/7',
        'Snap Fitness 24/7', 'David Lloyd Clubs', 'TrainMore Black Label',
        'SportCity Performance', 'High Studios', 'Saints & Stars Boutique', 'Bodytime EMS',
        'Fit For Free Premium', 'Fit20 Studio', 'Curves Women', 'RitualGym', 'Fresh Fitness',
        'Splash Healthclub', 'Fitland', 'Pathé Arena', 'Pathé City', 'Pathé De Kuip',
        'Pathé Schouwburgplein', 'Vue Alkmaar', 'Vue Eindhoven', 'Kinepolis Jaarbeurs',
        'Kinepolis Hoofddorp', 'GlowGolf', 'Jumpsquare', 'Jump XL', 'Bounz', 'Street Jump',
        'Monkey Town', 'Ballorig', 'Avontura', 'Spelekids', 'Play-In', 'Climb-Inn',
        'Roompot Servicepunt', 'Landal Servicepunt', 'Center Parcs Service Desk',
        'EuroParcs Servicepunt', 'TopParken Desk', 'Rooming House', 'Humphrey\'s', 'Gauchos',
        'La Cubanita', 'Spareribs Express', 'De Pizzabakkers', 'Mazara Pizzeria', 'Poke Perfect Express',
        'Wok To Go Express', 'Sushi Factory', 'Sushi Koi', 'Genki Sushi', 'Mr. Sushi',
        'Sushi Daily', 'Kippie', 'Slagerij Kippie', 'Keurslager Vers', 'Vlaams Friteshuis',
        'The Counter Burger', 'Bram Ladage', 'Frites Atelier', 'Vegan Junk Food Bar',
        'Flower Burger', 'BackFactory', 'Subway Fresh Forward', 'McCafé', 'Burger King Café',
        'KFC Drive', 'Domino\'s Pizza Theater', 'New York Pizza Slice', 'Pizza Hut Express',
        'Papa John\'s Express', 'Starbucks Drive Thru', 'Coffeecompany Compact',
        'Bagels & Beans Express', 'Anne&Max Express', 'Delifrance Station', 'Julia\'s To Go',
        'AH to go Station', 'HEMA Bakery Café', 'Partou Kinderopvang', 'KinderRijk',
        'Smallsteps', 'Humankind', 'Korein Kinderplein', 'Kindergarden', 'Compananny',
        'Up Kinderopvang', 'Skids', 'Lyceo', 'StudyWorks', 'BijlesAanHuis.nl', 'StudentsPlus',
        'Dag en Nacht Apotheek', 'DC Klinieken', 'Bergman Clinics', 'Eyescan', 'Optical Center',
        'Skin Clinics', 'Velthuis Kliniek', 'Sanquin Servicepunt', 'HairClinic', 'Rob Peetoom ColorBar',
        'Team Kappers Color', 'BrainWash Express', 'Kinki Color', 'Q-Park', 'APCOA Parking',
        'Interparking', 'P1 Parking', 'ParkBee', 'EasyPark Servicepunt', 'Greenwheels Servicepunt',
        'MyWheels Servicepunt', 'SnappCar Service Hub', 'Felyx Servicepunt', 'GO Sharing Servicepunt',
        'Check Servicepunt', 'Bolt Servicepunt', 'Uber Greenlight Hub', 'NS Servicewinkel',
        'Arriva Servicewinkel', 'Keolis Servicewinkel', 'Connexxion Servicepunt', 'RET Servicepunt',
        'GVB Service & Tickets', 'HTM Servicepunt', 'TUI at Home', 'KLM Ticket Office',
        'Transavia Desk', 'Ryanair Desk', 'easyJet Sales', 'Rituals Spa', 'Manicare',
        'Soap Treatment Store', 'Skins Cosmetics', 'Skins Spa', 'Babassu', 'Douglas Spa',
        'Etos Clinic', 'HEMA City', 'HEMA Beauty Lab', 'Suitsupply Custom Made',
        'VanHaren Custom', 'Nelson Premium', 'Manfield Made to Order', 'Sacha Limited',
        'Oger', 'The Society Shop', 'State of Art', 'Profuomo Store', 'NZA New Zealand Auckland',
        'Bolia', 'Ligne Roset', 'Montis Store', 'Arco Store', 'BoConcept', 'Hülsta Studio',
        'Keukenzaak Mandemakers', 'Bruynzeel Keukens', 'KeukenConcurrent', 'Grando Keukens',
        'I-Kook', 'Keukenloods', 'Keuken Kampioen', 'Tegeldepot', 'TegelMegaStore', 'Sanidirect',
        'Sanitairwinkel', 'Sanisale', 'Badkamerxxl Store', 'Keukenmaxx', 'Keukenwarenhuis',
        'Tegeldepot XL', 'PontMeyer Keukens', 'Karwei Studio', 'Gamma Keukens', 'Praxis Keukens',
        'Hornbach Projectbouw', 'Bauhaus Keukenwereld'
    ]
    
    # Normaliseer ketennamen (lowercase, strip whitespace)
    grote_ketens_normalized = [keten.lower().strip() for keten in grote_ketens]
    
    # Normaliseer bedrijfsnamen
    combined_df['name_normalized'] = combined_df['name'].astype(str).str.lower().str.strip()
    
    # Filter: behoud alleen bedrijven die NIET in de grote ketens lijst zitten
    # We controleren of de genormaliseerde naam een substring match heeft met een keten
    is_keten = combined_df['name_normalized'].apply(
        lambda name: any(keten in name for keten in grote_ketens_normalized)
    )
    
    print(f"Originele dataset: {len(combined_df)} bedrijven")
    print(f"Aantal grote ketens uitgesloten: {is_keten.sum()}")
    print(f"Bedrijven na filtering: {len(combined_df[~is_keten])}")
    
    # Toon voorbeelden van uitgesloten ketens
    if is_keten.sum() > 0:
        print(f"\nVoorbeelden van uitgesloten ketens:")
        excluded = combined_df[is_keten]['name'].head(10)
        for i, naam in enumerate(excluded, 1):
            print(f"  {i:2d}. {naam}")
    
    # Update combined_df
    combined_df = combined_df[~is_keten].copy()
    combined_df = combined_df.drop('name_normalized', axis=1)
    
else:
    print("Geen gecombineerde data beschikbaar voor ketens filtering!")


Geen gecombineerde data beschikbaar voor ketens filtering!


In [6]:
if 'combined_df' in locals():
    print("=== POSTCODE FILTERING (optioneel) ===")
    
    # Check of address kolom bestaat
    if 'address' in combined_df.columns:
        print("Filteren op basis van Nederlandse postcode...")
        
        # Filter op bedrijven met Nederlandse postcode (4 cijfers + 2 letters)
        postcode_pattern = r'\b\d{4}\s?[A-Z]{2}\b'
        postcode_mask = combined_df['address'].str.contains(postcode_pattern, case=False, na=False)
        filtered_df = combined_df[postcode_mask].copy()
        
        print(f"Originele dataset: {len(combined_df)} bedrijven")
        print(f"Met Nederlandse postcode: {len(filtered_df)} bedrijven")
        print(f"Verwijderde {len(combined_df) - len(filtered_df)} bedrijven zonder postcode")
        
        # Toon voorbeelden van adressen met postcode
        print(f"\nVoorbeelden van adressen met Nederlandse postcode:")
        sample_addresses = filtered_df['address'].dropna().head(10)
        for i, address in enumerate(sample_addresses, 1):
            print(f"  {i:2d}. {address}")
        
        # Update combined_df voor verdere verwerking
        combined_df = filtered_df.copy()
        
    else:
        print("Geen 'address' kolom gevonden!")
        print("Beschikbare kolommen:", list(combined_df.columns))
        
else:
    print("Geen gecombineerde data beschikbaar voor filtering!")


Geen gecombineerde data beschikbaar voor filtering!


## 3. Duplicaten Verwijderen

### Wat gebeurt er hier?
We verwijderen duplicaten op basis van bedrijfsnaam. Dit is belangrijk omdat bedrijven mogelijk in meerdere CSV bestanden voorkomen. We behouden de eerste voorkoming van elke unieke bedrijfsnaam.

**Duplicaten detectie:**
- ✅ Op basis van bedrijfsnaam (lowercase, gestript)
- ✅ GPS coördinaten worden genegeerd voor duplicaten detectie


In [7]:
if 'combined_df' in locals():
    print("=== DUPLICATEN VERWIJDEREN ===")
    
    # Maak een unieke key op basis van bedrijfsnaam (lowercase, gestript)
    combined_df['unique_key'] = combined_df['name'].astype(str).str.lower().str.strip()
    
    # Tel duplicaten
    duplicates_count = combined_df['unique_key'].duplicated().sum()
    print(f"Gevonden duplicaten: {duplicates_count}")
    
    # Verwijder duplicaten (behoud eerste voorkomen)
    deduplicated_df = combined_df.drop_duplicates(subset=['unique_key'], keep='first')
    
    print(f"Na verwijdering duplicaten: {len(deduplicated_df)} unieke bedrijven")
    print(f"Verwijderde {len(combined_df) - len(deduplicated_df)} duplicaten")
    
    # Verwijder de unique_key kolom
    deduplicated_df = deduplicated_df.drop('unique_key', axis=1)
    
else:
    print("Geen gefilterde data beschikbaar voor duplicaten verwijdering!")


Geen gefilterde data beschikbaar voor duplicaten verwijdering!


## 4. Data Kwaliteit Analyse - Ontbrekende data

### Wat gebeurt er hier?
We analyseren de kwaliteit van onze dataset door te kijken naar ontbrekende data. Dit helpt ons te begrijpen welke informatie we hebben en welke we missen.

**Analyse onderdelen:**
- ✅ Ontbrekende data per kolom
- ✅ Specifieke analyse voor adres gegevens
- ✅ Nederlandse postcode validatie
- ✅ Straatnaam detectie


In [8]:
if 'deduplicated_df' in locals():
    print("=== DATA KWALITEIT ANALYSE ===")
    
    # Basis statistieken
    print(f"Totaal aantal bedrijven: {len(deduplicated_df)}")
    print(f"Aantal kolommen: {len(deduplicated_df.columns)}")
    
    # Ontbrekende data per kolom
    print("\nOntbrekende data per kolom:")
    missing_data = deduplicated_df.isnull().sum()
    missing_percent = (missing_data / len(deduplicated_df)) * 100
    
    for col in deduplicated_df.columns:
        if missing_data[col] > 0:
            print(f"  {col}: {missing_data[col]} ({missing_percent[col]:.1f}%)")
        else:
            print(f"  {col}: 0 (0.0%)")
    
    # Specifieke analyse voor address kolom
    if 'address' in deduplicated_df.columns:
        print(f"\n=== ADRES ANALYSE ===")
        
        # Nederlandse postcode pattern
        postcode_pattern = r'\b\d{4}\s?[A-Z]{2}\b'
        has_postcode = deduplicated_df['address'].str.contains(postcode_pattern, na=False)
        
        # Straat naam pattern (uitgebreid)
        street_pattern = r'\b[A-Za-z][A-Za-z\s]*(?:straat|laan|weg|plein|park|dreef|kade|gracht|singel|boulevard|pad|steeg|hof|plantsoen|ring|baan|dijk|wal|haven|kanaal|brug|tunnel|viaduct|rotonde|kruispunt)\b'
        has_street = deduplicated_df['address'].str.contains(street_pattern, na=False)
        
        print(f"Met adres: {deduplicated_df['address'].notna().sum()} ({deduplicated_df['address'].notna().mean()*100:.1f}%)")
        print(f"Met Nederlandse postcode: {has_postcode.sum()} ({has_postcode.mean()*100:.1f}%)")
        print(f"Met straatnaam: {has_street.sum()} ({has_street.mean()*100:.1f}%)")
        
        # Voorbeelden van adressen zonder postcode
        no_postcode = deduplicated_df[deduplicated_df['address'].notna() & ~has_postcode]
        if len(no_postcode) > 0:
            print(f"\nVoorbeelden van adressen zonder Nederlandse postcode:")
            for i, address in enumerate(no_postcode['address'].head(5), 1):
                print(f"  {i}. {address}")
else:
    print("Geen gededupliceerde data beschikbaar voor analyse!")


Geen gededupliceerde data beschikbaar voor analyse!


## 5. Data Kwaliteit Heatmap

### Wat gebeurt er hier?
We maken visuele representaties van de data kwaliteit om snel te kunnen zien waar informatie ontbreekt. Dit helpt bij het identificeren van patronen in ontbrekende data.

**Visualisaties:**
- ✅ Heatmap van ontbrekende data voor eerste 100 bedrijven
- ✅ Bar chart van ontbrekende data percentages per veld


In [9]:
if 'deduplicated_df' in locals():
    print("=== DATA KWALITEIT HEATMAP ===")
    
    # Maak een subset van de data voor de heatmap (eerste 100 bedrijven)
    sample_size = min(100, len(deduplicated_df))
    sample_df = deduplicated_df.head(sample_size)
    
    # Maak een boolean matrix van ontbrekende data
    missing_matrix = sample_df.isnull()
    
    # Maak de heatmap
    plt.figure(figsize=(15, 8))
    sns.heatmap(missing_matrix.T, cbar=True, yticklabels=True, xticklabels=False, 
                cmap='RdYlBu_r', cbar_kws={'label': 'Ontbrekende data'})
    plt.title(f'Data Kwaliteit Heatmap (eerste {sample_size} bedrijven)')
    plt.xlabel('Bedrijven')
    plt.ylabel('Kolommen')
    plt.tight_layout()
    plt.show()
    
    # Maak een bar chart van ontbrekende data percentages
    plt.figure(figsize=(12, 6))
    missing_percent = (deduplicated_df.isnull().sum() / len(deduplicated_df)) * 100
    missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=True)
    
    if len(missing_percent) > 0:
        missing_percent.plot(kind='barh')
        plt.title('Percentage ontbrekende data per kolom')
        plt.xlabel('Percentage ontbrekend')
        plt.tight_layout()
        plt.show()
    else:
        print("Alle kolommen hebben complete data!")
else:
    print("Geen gededupliceerde data beschikbaar voor heatmap!")


Geen gededupliceerde data beschikbaar voor heatmap!


## 5. Filteren op complete adres data

### Wat gebeurt er hier?
We filteren de dataset om alleen bedrijven te behouden die een compleet adres hebben met zowel een straatnaam als een Nederlandse postcode. Dit zorgt voor een dataset met betrouwbare en complete locatie-informatie.

**Filter criteria:**
- ✅ Adres is niet leeg
- ✅ Bevat Nederlandse postcode (1234 AB format)
- ✅ Bevat straatnaam (straat, laan, weg, plein, etc.)


In [10]:
if 'deduplicated_df' in locals() and 'address' in deduplicated_df.columns:
    print("=== FILTEREN OP ADRES MET STRAAT EN POSTCODE ===")
    
    # Nederlandse postcode pattern
    postcode_pattern = r'\b\d{4}\s?[A-Z]{2}\b'
    
    # Filter criteria
    has_address = ~deduplicated_df['address'].isnull() & (deduplicated_df['address'] != '')
    has_postcode = deduplicated_df['address'].str.contains(postcode_pattern, na=False)
    
    # Straat check: simpelweg checken of er letters in het adres staan (maar niet alleen in de postcode)
    # Pak alles VOOR de postcode en check of daar letters in zitten
    # Dit werkt voor: "Hooghout 60 4817ED Breda" -> "Hooghout 60" heeft letters
    # Maar NIET voor: "4817ED Breda" (geen tekst voor postcode)
    
    # Split op postcode
    parts = deduplicated_df['address'].str.split(postcode_pattern, expand=True, regex=True)
    part_before_postcode = parts[0].fillna('')
    has_street = part_before_postcode.str.contains(r'[A-Za-z]{2,}', na=False)
    
    # Gefilterde dataset = heeft adres EN postcode EN straatnaam (tekst voor de postcode)
    valid_address = has_address & has_postcode & has_street
    
    print(f"Totaal bedrijven: {len(deduplicated_df)}")
    print(f"Met adres: {has_address.sum()} ({has_address.mean()*100:.1f}%)")
    print(f"Met postcode: {has_postcode.sum()} ({has_postcode.mean()*100:.1f}%)")
    print(f"Met straat: {has_street.sum()} ({has_street.mean()*100:.1f}%)")
    print(f"Met ADRES EN POSTCODE EN STRAAT: {valid_address.sum()} ({valid_address.mean()*100:.1f}%)")
    
    # Maak gefilterde dataset
    filtered_df = deduplicated_df[valid_address].copy()
    
    print(f"\nGefilterde dataset: {len(filtered_df)} bedrijven met compleet adres")
    
    # Toon voorbeelden van gefilterde adressen
    print("\n=== VOORBEELDEN VAN GEFILTERDE ADRESSEN ===")
    sample_addresses = filtered_df['address'].dropna().head(10)
    for i, address in enumerate(sample_addresses, 1):
        print(f"{i:2d}. {address}")
    
    # Toon voorbeelden van adressen die werden uitgesloten
    excluded_df = deduplicated_df[~valid_address]
    if len(excluded_df) > 0:
        print(f"\n=== VOORBEELDEN VAN UITGESLOTEN ADRESSEN ===")
        print("Adressen zonder straat of postcode:")
        sample_excluded = excluded_df['address'].dropna().head(5)
        for i, address in enumerate(sample_excluded, 1):
            print(f"{i:2d}. {address}")
        
else:
    print("Geen address kolom gevonden of geen data beschikbaar!")


Geen address kolom gevonden of geen data beschikbaar!


## 6. Categorieën en Amenities Analyse

### Wat gebeurt er hier?
We analyseren de verdeling van bedrijfscategorieën en amenities in onze gefilterde dataset. Dit geeft inzicht in welke soorten bedrijven we hebben gevonden en hoe ze zijn gecategoriseerd.

**Analyse onderdelen:**
- ✅ Identificatie van categorie kolommen
- ✅ Top categorieën met aantallen en percentages
- ✅ Bar charts en pie charts voor visualisatie
- ✅ Amenities breakdown voor complexe categorieën


In [11]:
if 'filtered_df' in locals():
    print("=== CATEGORIEËN EN AMENITIES ANALYSE ===")
    
    # Zoek naar categorie/amenity kolommen
    category_cols = []
    for col in filtered_df.columns:
        if any(keyword in col.lower() for keyword in ['category', 'amenity', 'type', 'tag', 'class']):
            category_cols.append(col)
    
    print(f"Gevonden categorie kolommen: {category_cols}")
    
    if category_cols:
        # Analyseer elke categorie kolom
        for col in category_cols:
            print(f"\n=== ANALYSE VAN {col.upper()} ===")
            
            # Tel unieke categorieën
            unique_categories = filtered_df[col].nunique()
            total_businesses = len(filtered_df)
            
            print(f"Totaal unieke categorieën: {unique_categories}")
            print(f"Totaal bedrijven: {total_businesses}")
            
            # Toon top categorieën
            category_counts = filtered_df[col].value_counts()
            
            # Check of er categorieën zijn
            if len(category_counts) == 0:
                print("\nGeen categorieën gevonden in de data!")
                continue
                
            print(f"\nTop 15 categorieën:")
            for i, (category, count) in enumerate(category_counts.head(15).items(), 1):
                percentage = (count / total_businesses) * 100
                print(f"  {i:2d}. {category}: {count} ({percentage:.1f}%)")
            
            # Maak visualisaties (alleen als er data is)
            if len(category_counts) > 0:
                try:
                    plt.figure(figsize=(15, 8))
                    
                    # Bar chart voor top 15 categorieën
                    plt.subplot(2, 1, 1)
                    top_categories = category_counts.head(min(15, len(category_counts)))
                    if len(top_categories) > 0:
                        top_categories.plot(kind='bar')
                        plt.title(f'Top {len(top_categories)} {col} categorieën')
                        plt.xlabel('Categorie')
                        plt.ylabel('Aantal bedrijven')
                        plt.xticks(rotation=45, ha='right')
                        plt.tight_layout()
                    
                    # Pie chart voor top 10 categorieën
                    plt.subplot(2, 1, 2)
                    top_10 = category_counts.head(min(10, len(category_counts)))
                    if len(top_10) > 0:
                        others = category_counts.iloc[min(10, len(category_counts)):].sum()
                        if others > 0:
                            pie_data = list(top_10.values) + [others]
                            pie_labels = list(top_10.index) + ['Overige']
                        else:
                            pie_data = top_10.values
                            pie_labels = top_10.index
                        
                        plt.pie(pie_data, labels=pie_labels, autopct='%1.1f%%', startangle=90)
                        plt.title(f'Verdeling van top {len(top_10)} {col} categorieën')
                    
                    plt.tight_layout()
                    plt.show()
                except Exception as e:
                    print(f"\nFout bij maken van visualisatie: {e}")
                    print("Visualisaties worden overgeslagen vanwege data probleem.")
            
            # Amenities Breakdown (als er kolommen zijn met dubbele punten)
            if ':' in str(filtered_df[col].dropna().iloc[0] if len(filtered_df) > 0 else ''):
                print(f"\n=== AMENITIES BREAKDOWN ===")
                
                # Split categorieën op dubbele punt
                all_amenities = []
                for categories in filtered_df[col].dropna():
                    if isinstance(categories, str):
                        amenities = [cat.strip() for cat in categories.split(':')]
                        all_amenities.extend(amenities)
                
                # Tel amenities
                amenity_counts = pd.Series(all_amenities).value_counts()
                print(f"Totaal unieke amenities: {len(amenity_counts)}")
                
                print(f"\nTop 15 amenities:")
                for i, (amenity, count) in enumerate(amenity_counts.head(15).items(), 1):
                    percentage = (count / len(all_amenities)) * 100
                    print(f"  {i:2d}. {amenity}: {count} ({percentage:.1f}%)")
                
                # Visualisatie van amenities
                plt.figure(figsize=(12, 6))
                top_amenities = amenity_counts.head(15)
                top_amenities.plot(kind='bar')
                plt.title('Top 15 Amenities')
                plt.xlabel('Amenity')
                plt.ylabel('Aantal voorkomens')
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                plt.show()
    
    else:
        print("Geen categorie kolommen gevonden!")
        print("Beschikbare kolommen:", list(filtered_df.columns))
        
else:
    print("Geen gefilterde data beschikbaar voor categorieën analyse!")


Geen gefilterde data beschikbaar voor categorieën analyse!


## 7. Export - Externe Opslag van Cleaned Data

### Wat gebeurt er hier?
We exporteren onze gefilterde en schone dataset naar een nieuwe CSV bestand in de `cleaned_data/` folder. Dit bestand kan worden gebruikt als input voor de scraper om duplicaten te voorkomen.

**Export details:**
- ✅ Timestamped bestand voor historie
- ✅ `_latest.csv` backup voor gemakkelijke toegang
- ✅ Originele bestanden blijven volledig onbeschadigd
- ✅ Source_file kolom wordt verwijderd uit export


In [12]:
if 'filtered_df' in locals():
    print("=== EXPORT NAAR CLEANED_DATA FOLDER ===")
    print("BELANGRIJK: Originele CSV bestanden worden NIET gewijzigd!")
    print("Alleen nieuwe bestanden worden aangemaakt in cleaned_data/ folder")
    
    # Maak cleaned_data directory als deze niet bestaat
    output_dir = Path('cleaned_data')
    output_dir.mkdir(exist_ok=True)
    
    # Timestamp voor bestandsnaam
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Export paths
    timestamped_file = output_dir / f'tilburg_bedrijven_cleaned_{timestamp}.csv'
    latest_file = output_dir / 'tilburg_bedrijven_cleaned_latest.csv'
    
    # Verwijder source_file kolom voor de export (niet nodig in output)
    export_df = filtered_df.copy()
    if 'source_file' in export_df.columns:
        export_df = export_df.drop('source_file', axis=1)
    
    # Export naar CSV
    export_df.to_csv(timestamped_file, index=False)
    export_df.to_csv(latest_file, index=False)
    
    print(f"Gefilterde data geëxporteerd naar:")
    print(f"  - {timestamped_file} ({len(export_df)} bedrijven)")
    print(f"  - {latest_file} (backup)")
    
    # Samenvatting statistieken
    print(f"\n=== EXPORT STATISTIEKEN ===")
    print(f"Totaal bedrijven geëxporteerd: {len(export_df)}")
    print(f"Kolommen in export: {len(export_df.columns)}")
    print(f"Kolommen: {list(export_df.columns)}")
    
    # Data loss overzicht
    print(f"\n=== DATA PIPELINE SAMENVATTING ===")
    if 'all_dataframes' in locals():
        original_count = sum(len(df) for df in all_dataframes)
        print(f"1. Originele data geladen: {original_count} bedrijven")
    if 'combined_df' in locals():
        print(f"2. Na filtering: {len(combined_df)} bedrijven")
    if 'deduplicated_df' in locals():
        print(f"3. Na duplicaten verwijdering: {len(deduplicated_df)} bedrijven")
    print(f"4. Na compleet adres filtering: {len(filtered_df)} bedrijven")
    print(f"\nFinale dataset: {len(export_df)} schone, unieke bedrijven met compleet adres")
    print("\nVEILIGHEID: Alle originele CSV bestanden blijven volledig onbeschadigd!")
    
else:
    print("Geen gefilterde data beschikbaar voor export!")


Geen gefilterde data beschikbaar voor export!


## 8. Samenvatting en Conclusies

### Wat hebben we bereikt?
Deze notebook heeft een complete data pipeline uitgevoerd om alle Tilburg bedrijfsdata te combineren, te filteren en te analyseren.

### Belangrijkste resultaten:
- **Data combinatie**: Alle CSV bestanden zijn gecombineerd zonder originele bestanden te wijzigen
- **Geografische filtering**: Alleen bedrijven in Tilburg zijn behouden
- **Duplicaten verwijdering**: Unieke bedrijven op basis van naam
- **Adres kwaliteit**: Alleen bedrijven met compleet adres (straat + postcode)
- **Categorieën analyse**: Inzicht in bedrijfstypen en amenities
- **Export**: Schone dataset voor gebruik in scraper

### Output bestanden:
- `cleaned_data/tilburg_bedrijven_cleaned_YYYYMMDD_HHMMSS.csv` - Timestamped export
- `cleaned_data/tilburg_bedrijven_cleaned_latest.csv` - Meest recente export

### Volgende stappen:
1. Gebruik `cleaned_data/tilburg_bedrijven_cleaned_latest.csv` als `--skip-csv` parameter in de scraper
2. Dit voorkomt dat de scraper bedrijven opnieuw bezoekt die al zijn gevonden
3. De scraper kan zich focussen op nieuwe bedrijven die nog niet zijn gescraped

### Data kwaliteit garantie:
- ✅ Alle bedrijven zijn in Tilburg gevestigd
- ✅ Alle bedrijven hebben een compleet adres met straat en postcode
- ✅ Duplicaten zijn verwijderd
- ✅ Originele CSV bestanden blijven volledig onbeschadigd
